# Adaptive Circuit Constrction

Mizore provides a framework for adaptive VQE like what is described in [J. Chem. Theory Comput. 2020, 16, 2](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.9b01084) and [Nat Commun 10, 3007 (2019)](https://www.nature.com/articles/s41467-019-10988-2), where the structure of the parameterized quantum circuits is also optimized, differing from traditional VQE which uses a fixed parameterized circuit and only varies the parameter. While providing better performance of convergence, adaptive method can also achieve certain objective with fewer quantum gates. We believe that adaptive circuit construction is a key method for near-term quantum applications.


## Introduction

In adaptive circuit construction algorithms, ansatz circuits are adaptively constructed by selecting and adding blocks from a predefined pool $\mathcal{E}$.

Specifically, in each iteration, we do the following steps
- Go over the blocks in $\mathcal{E}$;
- Calculate a score of every block based on certain criterion;
- Add the entangler with highest score to the circuit.

We believe circuit construction is important for *near-term* quantum computing because their adaptiveness can help them place the entangling gates in a more efficient way than the methods use fixed circuits. The two-qubit gate advantage of including adaptiveness has been shown in some works.

To design a adaptive circuit construction algorithm, one needs to define
- The blocks pool $\mathcal{E}$;
- The criterion for scoring the blocks;
- Where to add the new block.

In Mizore, we provide a general framework `CircuitConstrutor` for design such a framework. 

Including
- Handy block pool tools (See [BlockPools](BlockPools.ipynb) for details);
- Modules for efficiently and parallelly calculate usually used scores;
- Extendable framework for define the way blocks are updated.

## Basic Usage

To begin with, we show how to start a simple circuit construction, in which
- New blocks are added to the end of the circuit;
- The blocks score is based on the energy they can decrease by adding it to the end of the circuit;
- Energy descent gradient is calculated first and the blocks with high gradient will be selected to do a parameter optimization;
- The energy descent presented in the parameter optimization is defined to be the score of a block.

This construction can be easily carried out with the class `GreedyConstructor`. Here we start from construct the problem Hamiltonian and block pool.

In [3]:
from CircuitConstructor import GreedyConstructor
from HamiltonianGenerator import make_example_LiH
from HamiltonianGenerator.FermionTransform import jordan_wigner
from PoolGenerator import BlockPool,quasi_imaginary_evolution_rotation_pool
# Generate the problem to solve
energy_obj=make_example_LiH()
# Generate the block pool
pool=BlockPool(quasi_imaginary_evolution_rotation_pool(energy_obj.hamiltonian))

Symmetry: Coov  is used when build the molecule.


A plain constructor can be constructed with default parameters as follows.

In [4]:
# A plain constructor with 4 processor
constructor=GreedyConstructor(energy_obj,pool) 

Constructor with various properties can be constructed by specify the parameters as follows. (Select one to run)

In [5]:
# A 5 processor constructor with a name
from ParallelTaskRunner import TaskManager
task_manager=TaskManager(n_processor=5,task_package_size=20)
constructor=GreedyConstructor(energy_obj,pool,task_manager=task_manager,project_name="LiH") 

In [ ]:
# A constructor with specified optimizer
from ParameterOptimizer import BasinhoppingOptimizer
optimizer=BasinhoppingOptimizer(random_initial=0.01,niter=5)
constructor=GreedyConstructor(energy_obj,pool,optimizer=optimizer)

In [6]:
# A constructor with specified initial circuit
from Blocks import BlockCircuit, HartreeFockInitBlock
bc=BlockCircuit(6)
bc.add_block(HartreeFockInitBlock([0,1]))
constructor=GreedyConstructor(energy_obj,pool,init_circuit=bc)

The construction will not start immediately after build the constructor. Use `execute_construction` to start the run. Figures of energy descent and time used, as well as log of the run will be stored in the path /mizore_result/`project_name`_`time`

In [6]:
constructor.execute_construction()

Here is GreedyConstructor
Project Name: Untitled_09-02-15h29m00s
Block Pool Size: 96
Initial Cost: -7.698446722392026
********The 1th Iteration*********


Single Block Optimize 47904: 100%|██████████| 5/5 [00:05<00:00,  1.00s/it]


Block added and shown below, cost now is: -7.730543067110322 Hartree
********New Circuit********
Block Num:2; Qubit Num:6
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0, 1]
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 3, 4, 5]; Pauli:YXZYYX
Doing global optimization on the new circuit
Global Optimized Cost: -7.730543067110323
Distance to target cost: 0.1454584247050743
Gate Usage: {'CNOT': 10, 'SingleRotation': 13, 'TimeEvolution': 0}
Cost list: [-7.698446722392026, -7.730543067110323]
********The 2th Iteration*********


Single Block Optimize 47904: 100%|██████████| 5/5 [00:09<00:00,  1.97s/it]


No trial circuit in the list provides a lower cost
Circuit update failed
Suggestion: 1.Use larger pool 2.Ground cost may have achieved
Final Cost: -7.730543067110323
********Final Circuit********
Block Num:2; Qubit Num:6
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0, 1]
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 3, 4, 5]; Pauli:YXZYYX


## Extensions

### Other constuctors

We also provide a different kind of constructor `FixedDepthSweepConstructor`, which update the blocks in the circuit in a *sweep* way. 
Specifically, that is, update the blocks in the circuit of indices from `sweep_start_position` to `n_max_block-1` again and agian after constructing a circuit with `n_max_block`blocks.

Our numerical experiments show that, in this way, the number of blocks needed to achieve certain accuracy can be reduced. We also conjecture that by doing so we can avoiding local minimum in the calculation.

The following is an example for conserving gates by sweeps. The sweeps make the calculation converges by just 6 blocks. In contrast, a plain construction needs 7 blocks.

In [9]:
from CircuitConstructor import FixedDepthSweepConstructor
from HamiltonianGenerator import make_example_H2,get_reduced_energy_obj_with_HF_init
from HamiltonianGenerator.FermionTransform import get_parity_transform,make_transform_spin_separating
from PoolGenerator import BlockPool,quasi_imaginary_evolution_rotation_pool

# Generate a symmetry reduced problem Hamiltonian
transform = make_transform_spin_separating(get_parity_transform(8),8)
energy_obj = make_example_H2(basis="6-31g", fermi_qubit_transform=transform)
energy_obj=get_reduced_energy_obj_with_HF_init(energy_obj,[3,7])

pool=BlockPool(quasi_imaginary_evolution_rotation_pool(energy_obj.hamiltonian))

constructor=FixedDepthSweepConstructor(energy_obj,pool,n_max_block=10,sweep_start_position=1,no_global_optimization=False)
constructor.execute_construction()

Symmetry: Dooh  is used when build the molecule.
Here is FixedDepthSweepConstructor
Project Name: Untitled_09-02-15h55m26s
Block Pool Size: 128
Initial Cost: -1.126755317196931
********The 1th Iteration*********


Single Block Optimize 21504: 100%|██████████| 7/7 [00:07<00:00,  1.13s/it]


Block added and shown below, cost now is: -1.1330348090231104 Hartree
********New Circuit********
Block Num:2; Qubit Num:6
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0, 1, 2]
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 3, 4, 5]; Pauli:YXXYXY
Doing global optimization on the new circuit
Global Optimized Cost: -1.1330348090231104
Distance to target cost: 0.01763773584868189
Gate Usage: {'CNOT': 10, 'SingleRotation': 16, 'TimeEvolution': 0}
Cost list: [-1.126755317196931, -1.1330348090231104]
********The 2th Iteration*********


Single Block Optimize 21504: 100%|██████████| 7/7 [00:13<00:00,  1.98s/it]


Block added and shown below, cost now is: -1.1384595126095975 Hartree
********New Circuit********
Block Num:3; Qubit Num:6
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0, 1, 2]
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 3, 4, 5]; Pauli:YXXYXY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4, 5]; Pauli:XYXXZ
Doing global optimization on the new circuit
Global Optimized Cost: -1.1384798796731002
Distance to target cost: 0.01219266519869211
Gate Usage: {'CNOT': 18, 'SingleRotation': 25, 'TimeEvolution': 0}
Cost list: [-1.126755317196931, -1.1330348090231104, -1.1384798796731002]
********The 3th Iteration*********


...

Block added and shown below, cost now is: -1.1418011843398181 Hartree
********New Circuit********
Block Num:5; Qubit Num:6
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0, 1, 2]
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 3, 4, 5]; Pauli:YXXYXY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4, 5]; Pauli:XYXXZ
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4]; Pauli:XYYY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4, 5]; Pauli:YZYXY
Doing global optimization on the new circuit
Global Optimized Cost: -1.14180220155159
Distance to target cost: 0.0088703433202022
Gate Usage: {'CNOT': 32, 'SingleRotation': 43, 'TimeEvolution': 0}
Cost list: [-1.126755317196931, -1.1330348090231104, -1.1384798796731002, -1.138518885537022, -1.14180220155159]
********The 5th Iteration*********


Single Block Optimize 21504: 100%|██████████| 7/7 [00:24<00:00,  3.50s/it]


Block added and shown below, cost now is: -1.147412839804141 Hartree
********New Circuit********
Block Num:6; Qubit Num:6
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0, 1, 2]
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 3, 4, 5]; Pauli:YXXYXY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4, 5]; Pauli:XYXXZ
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4]; Pauli:XYYY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4, 5]; Pauli:YZYXY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 3]; Pauli:YX
Doing global optimization on the new circuit
Global Optimized Cost: -1.1474832964960984
Distance to target cost: 0.0031892483756938983
Gate Usage: {'CNOT': 34, 'SingleRotation': 48, 'TimeEvolution': 0}
Cost list: [-1.126755317196931, -1.1330348090231104, -1.1384798796731002, -1.138518885537022, -1.14180220155159, -1.1474832964960984]
********The 6th Iteration*********


Single Block Optimize 21504: 100%|██████████| 7/7 [00:24<00:00,  3.49s/it]


Block added and shown below, cost now is: -1.1513177083960893 Hartree
********New Circuit********
Block Num:7; Qubit Num:6
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0, 1, 2]
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 3, 4, 5]; Pauli:YXXYXY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4, 5]; Pauli:XYXXZ
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4]; Pauli:XYYY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4, 5]; Pauli:YZYXY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 3]; Pauli:YX
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 3]; Pauli:XXYX
Doing global optimization on the new circuit
Global Optimized Cost: -1.1513570605561187
Distance to target cost: -0.000684515684326481
Gate Usage: {'CNOT': 40, 'SingleRotation': 57, 'TimeEvolution': 0}
Cost list: [-1.126755317196931, -1.1330348090231104, -1.1384798796731002, -1.138518885537022, -1.14180220155159, -1.1474832964960984, -1.1513570605561187]
Target cost achieved by 7  bl

### More than ground state energy

The circuit constructors in Mizore are design to process a general type of objectives, not only the energy. By defining subclasses of `Objective` and `Cost`, the users can implement adaptive construction for any objective for which a cost function can be defined. 

Here, we show how to use the circuit constructors to construct an autoencoder which can compress a 4-qubit mixed state into a 2-qubit mixed state.

In [8]:
from Blocks import BlockCircuit
from HamiltonianGenerator.FermionTransform import jordan_wigner
from HamiltonianGenerator import make_example_LiH
from CircuitConstructor import GreedyConstructor
from PoolGenerator import BlockPool,quasi_imaginary_evolution_rotation_pool,all_rotation_pool
from Objective import PurityObjective
import pickle
# Construct the states to compress
circuit_list=[]
for path in ["LiH_1.5_test.bc","LiH_1.3_test.bc"]:
    with open("Objective/"+path, "rb") as f:
        circuit_list.append(pickle.load(f))

# Define an objective for make qubit 0,1 as pure as possible
purity_obj=PurityObjective(circuit_list,[0,1])

# Run the construction for purity
pool=BlockPool(all_rotation_pool(6,4))
constructor=GreedyConstructor(purity_obj,pool,max_n_iter=10,gradient_screening_rate=0.02,project_name="LiH_purity")
constructor.execute_construction()

Here is GreedyConstructor
Project Name: LiH_purity_09-02-15h45m49s
Block Pool Size: 926
Initial Cost: -1.9468399555383198
********The 1th Iteration*********


Single Block Optimize 58288: 100%|██████████| 19/19 [00:20<00:00,  1.08s/it]


Block added and shown below, cost now is: -1.99024034024064 Hartree
********New Circuit********
Block Num:1; Qubit Num:6
Block list:
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 4, 5]; Pauli:XYYY
Doing global optimization on the new circuit
Global Optimized Cost: -1.99024034024064
Distance to target cost: 0.009759659759360106
Gate Usage: {'CNOT': 6, 'SingleRotation': 9, 'TimeEvolution': 0}
Cost list: [-1.9468399555383198, -1.99024034024064]
********The 2th Iteration*********


Single Block Optimize 58288: 100%|██████████| 19/19 [00:26<00:00,  1.40s/it]


Block added and shown below, cost now is: -1.99178379156986 Hartree
********New Circuit********
Block Num:2; Qubit Num:6
Block list:
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 4, 5]; Pauli:XYYY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4]; Pauli:YXXX
Doing global optimization on the new circuit
Global Optimized Cost: -1.9917837922664998
Distance to target cost: 0.00821620773350018
Gate Usage: {'CNOT': 12, 'SingleRotation': 18, 'TimeEvolution': 0}
Cost list: [-1.9468399555383198, -1.99024034024064, -1.9917837922664998]
********The 3th Iteration*********


...

Single Block Optimize 58288: 100%|██████████| 19/19 [00:38<00:00,  2.01s/it]


Block added and shown below, cost now is: -1.9943992823505 Hartree
********New Circuit********
Block Num:4; Qubit Num:6
Block list:
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 4, 5]; Pauli:XYYY
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4]; Pauli:YXXX
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 2, 5]; Pauli:YXXX
Type:RotationEntangler; Para Num:1; Qsubset:[0, 1, 3, 4]; Pauli:YXXX
Doing global optimization on the new circuit
Global Optimized Cost: -1.99439928245874
Distance to target cost: 0.005600717541260014
Gate Usage: {'CNOT': 24, 'SingleRotation': 36, 'TimeEvolution': 0}
Cost list: [-1.9468399555383198, -1.99024034024064, -1.9917837922664998, -1.9940937046838403, -1.99439928245874]
********The 5th Iteration*********


Gradient 58288:  48%|████▊     | 448/926 [01:01<00:45, 10.61it/s]Process TaskRunner-30:
Process TaskRunner-31:
Process TaskRunner-28:
Process TaskRunner-29:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zijian/Mizore/src/ParallelTaskRunner/_task_manager.py", line 34, in run
    result = task.run()
  File "/home/zijian/Mizore/src/ParallelTaskRunner/_gradient_task.py", line 26, in run
    cost_list[i]=(obj(x)-init_cost)/self.step_size
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zijian/Mizore/src/Objective/_purity_obj.py", line 55, in obj
    one_DMs = evaluate_ansatz_1DMs(parameter, pcircuit.n_qubit, pcircuit.ansatz)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/

KeyboardInterrupt: 

Gradient 58288:  48%|████▊     | 448/926 [01:20<00:45, 10.61it/s]